In [3]:
import pandas as pd

def FCFS(jobs):
    df = pd.DataFrame(jobs)
    df['Completion'] = df['Processing time'].cumsum()
    df['Tardiness'] = df.apply(lambda row: max(0, row['Completion'] - row['Due Date']), axis=1)
    mean_flow_time = df['Completion'].mean()
    avg_tardiness = df['Tardiness'].mean()
    num_tardy_jobs = df['Tardiness'].apply(lambda x: 1 if x > 0 else 0).sum()
    total_completion = df['Completion'].sum()
    total_tardiness = df['Tardiness'].sum()
    total_df = pd.DataFrame([{'Job': 'Total', 'Processing time': '-', 'Due Date': '-', 'Completion': total_completion, 'Tardiness': total_tardiness}])
    df = pd.concat([df, total_df], ignore_index=True).to_string(index=False)
    print(df)
    print("Mean flow time =", mean_flow_time)
    print("Average tardiness =", avg_tardiness)
    print("Number of tardy jobs =", num_tardy_jobs)
    return mean_flow_time, avg_tardiness, num_tardy_jobs

def SPT(jobs):
    df = pd.DataFrame(jobs)
    df = df.sort_values(by='Processing time')
    df['Completion'] = df['Processing time'].cumsum()
    df['Tardiness'] = df.apply(lambda row: max(0, row['Completion'] - row['Due Date']), axis=1)
    mean_flow_time = df['Completion'].mean()
    avg_tardiness = df['Tardiness'].mean()
    num_tardy_jobs = df['Tardiness'].apply(lambda x: 1 if x > 0 else 0).sum()
    total_completion = df['Completion'].sum()
    total_tardiness = df['Tardiness'].sum()
    total_df = pd.DataFrame([{'Job': 'Total', 'Processing time': '-', 'Due Date': '-', 'Completion': total_completion, 'Tardiness': total_tardiness}])
    df = pd.concat([df, total_df], ignore_index=True).to_string(index=False)
    print(df)
    print("Mean flow time =", mean_flow_time)
    print("Average tardiness =", avg_tardiness)
    print("Number of tardy jobs =", num_tardy_jobs)
    return mean_flow_time, avg_tardiness, num_tardy_jobs

def EDD(jobs):
    df = pd.DataFrame(jobs)
    df = df.sort_values(by='Due Date')
    df['Completion'] = df['Processing time'].cumsum()
    df['Tardiness'] = df.apply(lambda row: max(0, row['Completion'] - row['Due Date']), axis=1)
    mean_flow_time = df['Completion'].mean()
    avg_tardiness = df['Tardiness'].mean()
    num_tardy_jobs = df['Tardiness'].apply(lambda x: 1 if x > 0 else 0).sum()
    total_completion = df['Completion'].sum()
    total_tardiness = df['Tardiness'].sum()
    total_df = pd.DataFrame([{'Job': 'Total', 'Processing time': '-', 'Due Date': '-', 'Completion': total_completion, 'Tardiness': total_tardiness}])
    df = pd.concat([df, total_df], ignore_index=True).to_string(index=False)
    print(df)
    print("Mean flow time =", mean_flow_time)
    print("Average tardiness =", avg_tardiness)
    print("Number of tardy jobs =", num_tardy_jobs)
    return mean_flow_time, avg_tardiness, num_tardy_jobs

def CR(jobs):
    df = pd.DataFrame(jobs)
    # Initialize current time
    current_time = 0
    # Initialize a list to store scheduled jobs
    scheduled_jobs = []
    # Initialize an empty list to store job scheduling details
    schedule_data = []
    # Iterate until all jobs are scheduled
    while len(scheduled_jobs) < len(df):
        # Compute the CR values for all jobs
        df['CR'] = (df['Due Date'] - current_time) / df['Processing time']
        # Filter out the jobs that are already scheduled
        unscheduled_jobs = df[~df['Job'].isin(scheduled_jobs)]
        if not unscheduled_jobs.empty:
            # Select the job with the minimum CR
            next_job = unscheduled_jobs.loc[unscheduled_jobs['CR'].idxmin()]
            # Schedule the selected job
            next_job_start_time = max(current_time, next_job['Due Date'] - next_job['Processing time'])
            next_job_end_time = next_job_start_time + next_job['Processing time']
            scheduled_jobs.append(next_job['Job'])
            # Update the current time
            current_time = next_job_end_time
            # Calculate tardiness
            tardiness = max(0, next_job_end_time - next_job['Due Date'])
            # Add scheduling details to the list
            schedule_data.append({'Job': next_job['Job'],
                                  'Processing time': next_job['Processing time'],
                                  'Due Date': next_job['Due Date'],
                                  'Completion': next_job_end_time,
                                  'Tardiness': tardiness})
    # Add total row to the list
    total_tardiness = sum(entry['Tardiness'] for entry in schedule_data)
    total_completion_time = sum(entry['Completion'] for entry in schedule_data[:-1])
    schedule_data.append({'Job': 'Total',
                          'Processing time': '-',
                          'Due Date': '-',
                          'Completion': total_completion_time,
                          'Tardiness': total_tardiness})
    # Convert the list of dictionaries to a DataFrame
    schedule_df = pd.DataFrame(schedule_data)
    # Calculate mean flow time
    mean_flow_time = schedule_df['Completion'][:-1].mean()
    # Calculate average tardiness
    avg_tardiness = schedule_df['Tardiness'][:-1].mean()
    # Calculate number of tardy jobs
    num_tardy_jobs = (schedule_df['Tardiness'][:-1] > 0).sum()
    schedule_df = pd.DataFrame(schedule_data).to_string(index=False)
    print(schedule_df)
    print("Mean flow time =", mean_flow_time)
    print("Average tardiness =", avg_tardiness)
    print("Number of tardy jobs =", num_tardy_jobs)
    return mean_flow_time, avg_tardiness, num_tardy_jobs

# Example jobs data
jobs = [
    {'Job': '1', 'Processing time': 3, 'Due Date': 4},
    {'Job': '2', 'Processing time': 6, 'Due Date': 8},
    {'Job': '3', 'Processing time': 8, 'Due Date': 12},
    {'Job': '4', 'Processing time': 4, 'Due Date': 15},
    {'Job': '5', 'Processing time': 2, 'Due Date': 11},
    {'Job': '6', 'Processing time': 1, 'Due Date': 25},
    {'Job': '7', 'Processing time': 7, 'Due Date': 21}
]

# FCFS
print("First Come First Served (FCFS):")
fcfs_metrics = FCFS(jobs)
print()

# SPT
print("Shortest Processing Time (SPT):")
spt_metrics = SPT(jobs)
print()

# EDD
print("Earliest Due Date (EDD):")
edd_metrics = EDD(jobs)
print()

# CR
print("Critical Ratio Order (CR):")
cr_metrics = CR(jobs)
print()

# Create a summary DataFrame
summary_df = pd.DataFrame({
    'Rule': ['FCFS', 'SPT', 'EDD', 'CR'],
    'Mean Flow Time': [fcfs_metrics[0], spt_metrics[0], edd_metrics[0], cr_metrics[0]],
    'Average Tardiness': [fcfs_metrics[1], spt_metrics[1], edd_metrics[1], cr_metrics[1]],
    '# Tardy Jobs': [fcfs_metrics[2], spt_metrics[2], edd_metrics[2], cr_metrics[2]]
}).to_string(index=False)

print("\n\nSUMMARY:")
print(summary_df)


First Come First Served (FCFS):
  Job Processing time Due Date  Completion  Tardiness
    1              20       25          20          0
    2              14       45          34          0
    3              35       50          69         19
    4              10       30          79         49
Total               -        -         202         68
Mean flow time = 50.5
Average tardiness = 17.0
Number of tardy jobs = 2

Shortest Processing Time (SPT):
  Job Processing time Due Date  Completion  Tardiness
    4              10       30          10          0
    2              14       45          24          0
    1              20       25          44         19
    3              35       50          79         29
Total               -        -         157         48
Mean flow time = 39.25
Average tardiness = 12.0
Number of tardy jobs = 2

Earliest Due Date (EDD):
  Job Processing time Due Date  Completion  Tardiness
    1              20       25          20          0
    4   